---

# **Processamento de Linguagem Natural (2025-Q3)**

---

Prof. Alexandre Donizeti Alves






---

### **EQUIPE**

---

**INTEGRANTES DO GRUPO:**


**Integrante 01:**

- Vitor Vieira Fernandes
- 11202130698

**Integrante 02:**

- Douglas Rodrigues Costa
-

**Integrante 03:**

- Matheus Calixto Viana
-


---

### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---



>


**LLM**: Google Gemini (gemini-flash-latest)

>

**Link para a documentação oficial**: https://ai.google.dev/gemini-api/docs/models?hl=pt-br


---
### **API**
---


**API**: Steam User Reviews API (Parte da Steam Storefront API)

**Site oficial**: https://store.steampowered.com/

**Link para a documentação oficial**: https://partner.steamgames.com/doc/store/getreviews


---
### **DESCRIÇÃO**
---

Este projeto implementa um notebook no Google Colab que integra o framework LangChain com o LLM Google Gemini para realizar uma análise aprofundada de reviews de jogos feitas por usuários da plataforma Steam. O sistema processa o texto, e cria uma experiência multimídia ao converter a análise em áudio com entonação emocional.

O fluxo de trabalho consiste em:

- **Coleta de Dados:** Obtenção de reviews reais de jogos populares via Steam API.

- **Filtragem Inteligente:** Seleção aleatória de reviews de qualidade (com base no tamanho do texto) para evitar spam ou avaliações vazias.

- **Processamento de Linguagem Natural (PLN):** Aplicação de múltiplas técnicas para extrair inteligência do texto bruto.

Técnicas de PLN Utilizadas:

- **Análise de Sentimentos:** O modelo classifica a opinião do jogador como Positiva, Negativa ou Mista/Neutra, identificando a polaridade emocional do texto.

- **Sumarização de Textos e Extração de Tópicos:** O sistema gera uma "Breve Análise" do review e extrai explicitamente os "Pontos Positivos" e "Pontos Negativos" citados pelo jogador, estruturando dados não estruturados.

- **Geração de Fala (Text-to-Speech) com Emoção:** Como diferencial criativo, o projeto utiliza o modelo Gemini TTS para narrar o review original. A narração adapta o tom de voz (animado, crítico, neutro) automaticamente com base no sentimento detectado na etapa de análise.

Link para o repositório no GitHub:

https://github.com/vitimbro/projeto-processamento-linguagem-natural

-----

## 1\. Configuração do Ambiente e Chaves de API

---

Para que este notebook funcione, ele precisa se comunicar com a API do Google Gemini. Para isso, é necessário fornecer uma chave de API pessoal.

**Este notebook foi configurado para NÃO salvar sua chave diretamente no código, garantindo sua segurança.**

Siga os passos abaixo para configurar sua chave:

### Passo 1: Obtenha sua Chave da API do Gemini

1.  Acesse o **[Google AI Studio](https://makersuite.google.com/app/apikey)**.
2.  Faça login com sua conta Google.
3.  Clique em **"Create API key"** (Criar chave de API) para gerar uma nova chave.
4.  Copie a chave gerada. Ela é uma longa sequência de letras e números.

### Passo 2: Armazene a Chave no Colab Secrets

1.  Na barra lateral esquerda deste notebook, clique no ícone de chave ( **🔑** ) para abrir a aba de **"Secrets"** (Gerenciador de secrets).
2.  Clique em **"+ Adicionar novo secret"**.
3.  No campo `name` (nome), digite exatamente:
    ```
    GOOGLE_API_KEY
    ```
4.  No campo `value` (valor), cole a sua chave de API que você copiou do Google AI Studio.
5.  Ative o botão (slider) ao lado do nome para permitir que o notebook acesse este secret.

Pronto\! Agora, ao executar as células de código abaixo, ela irá carregar sua chave de forma segura sem nunca expô-la.



---




### 1.1 Instalação das Bibliotecas e Dependências

Esta célula é responsável por preparar o ambiente de desenvolvimento no Google Colab, instalando todas as ferramentas externas necessárias para o funcionamento do projeto.


- `google-generativeai`: O SDK oficial para acessar a API do Google Gemini.

- `langchain` e `langchain-google-genai`: O framework utilizado para orquestrar o fluxo da aplicação e integrar o LLM.

- `requests`: Biblioteca utilizada para fazer a conexão e baixar os dados da API da Steam.


In [1]:
%%capture
# 1.1 Instalações

# Esta célula instala todas as bibliotecas necessárias.

# NOTA: É comum o Colab gerar um AVISO sobre um "conflito de dependência"
# Este aviso é esperado e não impede a execução do projeto.

!pip install -q -U google-generativeai langchain-google-genai langchain requests



---




### 1.2 Importações e Configuração da API

Esta célula é o "motor de partida" do nosso projeto. Ela realiza duas funções críticas: carrega as ferramentas (bibliotecas) que vamos usar e autentica nossa conexão com a Inteligência Artificial do Google.

**Principais Ações do Código:**

- * `google.generativeai`: O SDK que nos permite conversar com o modelo Gemini.

- * `requests`: Permite que o Python "navegue na internet" para buscar os dados da Steam.

- * `google.colab.userdata`: Uma ferramenta específica do Colab para acessar dados sensíveis de forma segura.

**Segurança (Secrets)**:

- A linha `userdata.get('GOOGLE_API_KEY')` é crucial. Ela busca a chave que você salvou na aba "Secrets" do Colab.

- Isso garante que sua chave de API **nunca fique visível** no código, prevenindo o uso indevido por terceiros caso você compartilhe o notebook.


In [2]:
# 1.2 Importações e Configuração da API

import os
import requests
import google.generativeai as genai
from google.colab import userdata

from google.genai import types

print("Configurando a chave da API do Google...")
try:
    # Carrega a chave de API a partir do 'Secrets' do Colab
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ Chave da API configurada com sucesso!")
except Exception as e:
    print(f"🚨 Erro ao configurar a chave: {e}")
    print("Verifique se você criou o secret 'GOOGLE_API_KEY' corretamente.")

Configurando a chave da API do Google...
✅ Chave da API configurada com sucesso!




---




### 1.3 Teste de Conexão com o Gemini

Esta etapa é uma **verificação de segurança** (sanity check). Seu objetivo é confirmar se a chave de API fornecida é válida e se o notebook consegue se comunicar com os servidores do Google antes de iniciarmos a análise de dados.

* **Ping na API:** O código executa a função `genai.list_models()`. Esta função pede ao Google a lista de todos os modelos de IA disponíveis para a sua conta.

* **Saída Limpa:** Para evitar poluir a tela, o comando que imprimiria os nomes (`print(model.name)`) foi deixado como um **comentário** (`#`). O loop acontece apenas para testar a conexão.


In [14]:
# 1.3 Teste de Conexão com o Gemini

print("Testando a conexão com a API do Gemini...")
try:
    # O loop abaixo testa a autenticação.
    for model in genai.list_models():
        if 'generateContent' in model.supported_generation_methods:
            # Se quiser ver a lista de modelos disponíveis, descomente a linha abaixo:
            # print(model.name)
            pass

    print("\n✅ Conexão com o Gemini bem-sucedida!")
except Exception as e:
    print(f"🚨 Falha na conexão com o Gemini: {e}")

Testando a conexão com a API do Gemini...

✅ Conexão com o Gemini bem-sucedida!




---




### 1.4 Teste de Conexão com a API da Steam

Após confirmar o acesso à IA, esta célula verifica se conseguimos extrair dados da nossa fonte: a loja da Steam. Diferente do Gemini, esta é uma API pública que não requer chave de autenticação, mas exige uma formatação correta da URL.

**O que este código faz:**

* **Define um Alvo:** Utilizamos o ID `1086940` (do jogo *Baldur's Gate 3*) como caso de teste.
* **Configura a Requisição:** Monta a URL do endpoint `appreviews` e define parâmetros vitais:

    * `json=1`: Para receber os dados em formato estruturado.

    * `num_per_page=1`: Solicitamos apenas uma review para economizar banda neste teste.

    * `language='brazilian'`: Filtra apenas reviews em Português do Brasil.

* Se tudo der certo, imprime um trecho da review coletada, provando que temos acesso aos dados brutos de texto (nosso *corpus*) para a análise de PLN.

In [4]:
# 1.4 Teste de Conexão com a API da Steam

print("Testando a conexão com a API da Steam...")
# Usando o appid de 'Baldur's Gate 3' como exemplo
appid_teste = 1086940
url_steam = f"https://store.steampowered.com/appreviews/{appid_teste}"
params = {'json': 1, 'num_per_page': 1, 'language': 'brazilian'}

try:
    response = requests.get(url_steam, params=params)
    if response.status_code == 200:
        review_data = response.json()
        if review_data.get('success') == 1:
            print("✅ Conexão com a Steam bem-sucedida!")
            # Imprime um trecho da primeira review encontrada
            print("\nExemplo de review obtida:")
            print(review_data['reviews'][0]['review'][:300] + "...")
        else:
             print(f"🚨 A API da Steam respondeu com um erro: {review_data.get('query_summary')}")
    else:
        print(f"🚨 Erro na requisição HTTP: Status Code {response.status_code}")
except Exception as e:
    print(f"🚨 Falha na conexão com a Steam: {e}")

Testando a conexão com a API da Steam...
✅ Conexão com a Steam bem-sucedida!

Exemplo de review obtida:
Esse jogo é tão bom que eu tinha o pirata e fiz questão de comprar e rejogar ele. 
Isso não é um jogo, é uma obra de arte. 
Suga sua vida, você vai sonhar com a parada e pensar enquanto não tiver jogando. 
Acabei de zerar de novo e já to planejando minha próxima run.
Apenas joguem....


---
# 2. Obtenção de Dados via API
---


Esta célula define a função principal do projeto, `obter_reviews_steam`, que atua como a ponte entre o nosso código e a base de dados da Steam.

**Destaques da Implementação:**

* **Parâmetros da Requisição:** Configuramos a chamada à API com filtros específicos:

    * `filter='recent'`: Garante que estamos pegando opiniões atuais sobre o jogo.

    * `language='brazilian'`: Restringe a busca para reviews em **Português do Brasil**, facilitando a análise pelo LLM e a compreensão do público alvo.

    * `num_per_page`: Define quantos reviews queremos baixar (o padrão é 100).

* **Extração e Simplificação:**

    * O código recebe um JSON complexo da Steam contendo muitas informações (autor, tempo de jogo, votos, etc.).

    * Nesta versão simplificada, iteramos sobre os resultados e extraímos **apenas o texto da review** (`review_data['review']`), armazenando-o em uma lista de strings. Isso torna o processamento posterior mais leve.

* **Teste Imediato:** Ao final da célula, executamos um teste prático buscando reviews do jogo *Stardew Valley* (ID 413150). Isso serve para confirmar visualmente que a conexão com a API foi bem-sucedida e que os dados estão chegando corretamente.

In [5]:
# 2. Função para obter reviews da Steam (Versão Simplificada)

def obter_reviews_steam(appid, quantidade=100):
    """
    Busca uma quantidade maior de reviews recentes de um jogo na Steam.
    Retorna uma lista contendo apenas os textos das reviews.
    """
    print(f"Buscando um lote de até {quantidade} reviews para o jogo ID: {appid}...")
    url = f"https://store.steampowered.com/appreviews/{appid}"
    params = {
        'json': 1,
        'filter': 'recent',
        'language': 'brazilian',
        'num_per_page': quantidade, # Pede um número maior de reviews
        'review_type': 'all'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        lista_de_textos = [] # Retornará apenas os textos
        if data.get('success') == 1 and 'reviews' in data:
            for review_data in data['reviews']:
                # Adiciona apenas a string de texto da review
                lista_de_textos.append(review_data['review'])

            print(f"✅ {len(lista_de_textos)} reviews obtidas com sucesso!")
            return lista_de_textos
        else:
            print("🚨 A API da Steam não retornou reviews para este jogo.")
            return []
    except requests.exceptions.RequestException as e:
        print(f"🚨 Erro ao acessar a API da Steam: {e}")
        return []

# --- Exemplo de uso da função ---
# ID de Stardew Valley: 413150
id_do_jogo_alvo = 413150
lista_de_reviews = obter_reviews_steam(id_do_jogo_alvo, quantidade=5)

# Imprime um trecho da primeira review para verificar
if lista_de_reviews:
    print("\n--- Exemplo de Review ---")
    print(lista_de_reviews[0][:500] + "...")

Buscando um lote de até 5 reviews para o jogo ID: 413150...
✅ 5 reviews obtidas com sucesso!

--- Exemplo de Review ---
E liegal cara(a ultima parte ruim e que não podemos pegar casadas)...


---
# 3. Aplicação das Técnicas de PLN
---

### Configuração do LangChain e Definição das Tarefas

Esta célula é o "cérebro" do projeto. Aqui, integramos o poder do Google Gemini com a estrutura do framework LangChain para definir exatamente como as reviews devem ser analisadas.

**O que o código faz:**

1.  **Inicialização do Modelo (LLM):**

    * Utilizamos o `ChatGoogleGenerativeAI` para carregar o modelo **`gemini-flash-latest`**.

    * **Por que este modelo?** O modelo "Flash" é otimizado para velocidade e eficiência, possuindo uma cota gratuita muito maior (1500 requisições/minuto).

    * **`temperature=0.2`**: Definimos uma temperatura baixa. Isso torna o modelo mais "focado" e determinístico, reduzindo a criatividade excessiva (alucinações) e garantindo que ele extraia apenas o que está realmente no texto.

2.  **Engenharia de Prompt (Prompt Engineering):**

    * O `prompt_unico_template` é a instrução detalhada que enviamos à IA.

    * Unificamos as tarefas (Análise de Sentimentos + Extração de Aspectos) em um único pedido para economizar recursos.

    * **Regras Estritas:** Incluímos diretrizes como *"Seja Literal"* e *"Não faça suposições"* para garantir que a análise seja fiel à opinião do jogador.

    * **Formatação:** Instruímos o modelo a usar **negrito** nos títulos, facilitando a leitura da saída final.

3.  **Criação da "Chain" (Cadeia):**

    * A linha `chain_unica = prompt_unico | llm` utiliza a sintaxe moderna do LangChain (LCEL).

    * Ela conecta o nosso "molde" (o prompt) com o "motor" (o Gemini), criando uma ferramenta pronta para receber o texto de uma review e devolver a análise completa.

In [15]:
# 6. Configuração do LangChain e Definição das Tarefas

# Libs do LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# ==========================================
#    PARÂMETROS DE CONFIGURAÇÃO
# ==========================================

# Escolha o modelo (ex: "gemini-flash-latest", "gemini-pro-latest")
MODELO_ESCOLHIDO = "gemini-pro-latest"

# Temperatura (0.0 = mais preciso/robótico, 1.0 = mais criativo/aleatório)
TEMPERATURA = 0.2

# ==========================================

print(f"Inicializando o modelo '{MODELO_ESCOLHIDO}' com temperatura {TEMPERATURA}...")

# Configura a API-KEY para AMBAS as bibliotecas
try:
    API_KEY = os.environ['GOOGLE_API_KEY']
    genai.configure(api_key=API_KEY)
except KeyError:
    print("🚨 ERRO: Chave 'GOOGLE_API_KEY' não encontrada nos Secrets. Por favor, configure-a.")

# --- Inicialização do LLM usando os Parâmetros ---
llm = ChatGoogleGenerativeAI(model=MODELO_ESCOLHIDO,
                             temperature=TEMPERATURA)

print("✅ Modelo LLM pronto para uso!")
print("\nCriando o template de prompt único com Emoções de Ekman...")

# --- PROMPT ÚNICO E OTIMIZADO ---
prompt_unico_template = """
Sua tarefa é fazer uma análise completa da review de um jogo da Steam. Siga as regras e o formato de saída estritamente.

REGRAS:
1.  **Seja Literal:** Extraia apenas pontos positivos/negativos que foram EXPLICITAMENTE mencionados. NÃO faça suposições.
2.  **Seja Conciso:** A "Breve Análise" deve ser um parágrafo curto resumindo o review. Os pontos devem ser tópicos simples (use "-").
3.  **Seja Limpo:** Se não houver pontos positivos ou negativos, escreva "Nenhum ponto mencionado."
4.  **Seja Preciso:** A "Classificação de Sentimento" deve ser apenas uma das três opções: Positivo, Negativo, ou Misto/Neutro.
5.  **Identifique a Emoção:** A "Emoção Predominante" deve ser uma das 6 emoções básicas de Ekman: Felicidade, Tristeza, Raiva, Medo, Surpresa ou Nojo. Escolha a que melhor representa o tom do texto.

Review: "{texto_review}"

Breve Análise:
[Escreva um parágrafo curto aqui]

Pontos Positivos:
- [Liste os tópicos aqui]

Pontos Negativos:
- [Liste os tópicos aqui]

Classificação de Sentimento:
[Positivo, Negativo, ou Misto/Neutro]

Emoção Predominante:
[Felicidade, Tristeza, Raiva, Medo, Surpresa ou Nojo]
"""

# Cria a chain única que será usada
prompt_unico = PromptTemplate.from_template(prompt_unico_template)
chain_unica = prompt_unico | llm

print("✅ Prompt e Chain criados com sucesso!")

Inicializando o modelo 'gemini-pro-latest' com temperatura 0.2...
✅ Modelo LLM pronto para uso!

Criando o template de prompt único com Emoções de Ekman...
✅ Prompt e Chain criados com sucesso!


---
# 4. Funções Principais
---


### **4.1 Análise e Processamento da Review**

Esta célula consolida toda a lógica desenvolvida anteriormente em uma única função robusta chamada `analisar_review_steam`. O objetivo é tornar o código reutilizável, permitindo analisar diferentes jogos apenas chamando esta função com o ID desejado.

**O que esta função realiza:**

1.  **Integração Global:** Utiliza `global review_texto_global, sentimento_global` para exportar os resultados. Isso é essencial para que a **Célula de Áudio** (que vem depois) saiba qual texto ler e com qual emoção.

2.  **Coleta e Filtragem:**
    * Busca um lote de reviews (padrão: 100).
    * Aplica um **Filtro de Qualidade**: Descarta reviews muito curtas (menores que 30 caracteres) que geralmente não possuem conteúdo relevante para análise.

3.  **Seleção Aleatória:** Escolhe uma review aleatória da lista filtrada, garantindo que cada execução traga uma análise nova.

4.  **Inteligência Artificial:** Envia o texto para o **Gemini** (via LangChain) para gerar o resumo, os pontos positivos/negativos e a classificação.

5.  **Mineração de Dados (Regex):** Aplica a lógica robusta que desenvolvemos para encontrar a "Classificação de Sentimento" dentro da resposta do Gemini, limpando formatações como negrito (`*`) ou quebras de linha, para garantir que o tom de voz do áudio seja correto.

6.  **Exibição Rica:** Formata e exibe o resultado final usando Markdown, apresentando o texto original e a análise da IA de forma visualmente organizada.

In [16]:
import time
import random
import textwrap
from IPython.display import display, Markdown
import re

# Inicializa a nova variável global (caso ainda não exista)
emocao_global = ""

def analisar_review_steam(id_jogo, qtd_busca=100, min_len=30):
    """
    Busca reviews, filtra, seleciona uma aleatória, analisa com Gemini
    (Sentimento + Emoção) e exibe o resultado.
    """
    # Importante: Permite modificar as variáveis para a célula de áudio
    global review_texto_global, sentimento_global, emocao_global

    # --- OBTENÇÃO DOS DADOS ---
    todas_as_reviews = obter_reviews_steam(id_jogo, quantidade=qtd_busca)

    # --- FILTRAGEM ---
    reviews_filtradas = [
        texto for texto in todas_as_reviews
        if len(texto.strip()) >= min_len
    ]

    # --- VERIFICAÇÃO ---
    if not reviews_filtradas:
        print(f"🚨 Não foi possível encontrar reviews com mais de {min_len} caracteres para o jogo ID: {id_jogo}.")
        return

    # --- LÓGICA PRINCIPAL ---
    print(f"Buscando uma review aleatória de um total de {len(reviews_filtradas)} reviews filtradas...")

    # Seleção Aleatória
    review_texto = random.choice(reviews_filtradas)

    # Análise com Gemini
    resultado_completo_obj = chain_unica.invoke({"texto_review": review_texto})
    resultado_completo = resultado_completo_obj.content

    # --- EXTRAÇÃO DE DADOS ---
    review_texto_global = review_texto

    # Valores padrão caso o parser falhe
    sentimento_encontrado = "Misto/Neutro"
    emocao_encontrada = "Neutro"

    linhas = resultado_completo.split('\n')

    for i, linha in enumerate(linhas):
        linha_lower = linha.lower()

        # --- Extrai o Sentimento ---
        if "classificação de sentimento" in linha_lower:
            if i + 1 < len(linhas):
                # Tenta pegar na próxima linha (padrão do prompt)
                valor = re.sub(r'[\*]', '', linhas[i+1]).strip()
                if valor: sentimento_encontrado = valor

        # --- Extrai a Emoção ---
        if "emoção predominante" in linha_lower:
            if i + 1 < len(linhas):
                valor = re.sub(r'[\*]', '', linhas[i+1]).strip()
                if valor: emocao_encontrada = valor

    # Atualiza as globais
    sentimento_global = sentimento_encontrado
    emocao_global = emocao_encontrada

    # --- APRESENTAÇÃO ---
    markdown_output = (f"""
---
## Análise de Review Aleatória do Jogo: `{id_jogo}`

---

### 📝 Texto Original
> {review_texto.strip().replace(chr(10), '  '+chr(10))}

---

### 🤖 Resultados da Análise com Gemini

{resultado_completo.strip()}

---
**Resumo dos Dados Extraídos:**
* **Sentimento:** `{sentimento_global}`
* **Emoção:** `{emocao_global}`
""")

    print("✅ Análise concluída!")
    display(Markdown(markdown_output))


---

### **4.2 Geração de Áudio Emocional (TTS)**

Esta célula é responsável por transformar o texto da review e a análise de sentimento em um arquivo de áudio narrado. Ela utiliza o modelo `gemini-2.5-flash-preview-tts`, que é capaz de modular a entonação da voz com base em instruções textuais.

**Principais Funcionalidades:**

1.  **Função Helper `save_audio_file`**:

    * Recebe os dados brutos de áudio (PCM) gerados pela API.

    * Converte e salva esses dados em um arquivo `.wav` padrão (Mono, 24kHz), que pode ser reproduzido no notebook.

2.  **Função Principal `gerar_audio_review`**:

    * **Entrada de Dados**: Lê as variáveis globais `review_texto_global`, `sentimento_global` e `emocao_global` que foram preenchidas pela célula de análise anterior.

    * **Lógica de Persona de Voz**: Baseada na teoria das emoções básicas de Ekman, o código seleciona dinamicamente uma voz e um tom apropriado:
        * *Felicidade* -> Voz **Puck** (Otimista)
        * *Raiva* -> Voz **Fenrir** (Agitada)
        * *Tristeza* -> Voz **Schedar** (Grave/Lenta)
        * *Surpresa* -> Voz **Autonoe** (Brilhante)
        * *Medo* -> Voz **Enceladus** (Suave)
        * *Nojo* -> Voz **Algenib** (Rouca)

    * **Prompt de Áudio**: Constrói uma instrução para o modelo, ex: *"Leia a seguinte review de jogo de forma triste, desanimado e lento: [Texto da Review]"*.

    * **Configuração da API**: Envia a requisição para o Google Gemini, especificando o modelo de voz escolhido.

    * **Reprodução**: Se a geração for bem-sucedida, exibe um player de áudio interativo (`IPython.display.Audio`) para que o usuário possa ouvir o resultado imediatamente.

In [20]:
import wave
from IPython.display import Audio, display
import os
import google.generativeai as genai
from google.genai import types

# --- FUNÇÃO HELPER: SALVAR ARQUIVO ---
def save_audio_file(audio_data, filename="review_audio.wav"):
    """Salva os dados de áudio brutos (PCM) em um arquivo .wav."""
    try:
        with wave.open(filename, "wb") as wf:
            wf.setnchannels(1)       # Mono
            wf.setsampwidth(2)       # 16-bit PCM
            wf.setframerate(24000)   # 24kHz (Padrão Gemini)
            wf.writeframes(audio_data)
        return True
    except Exception as e:
        print(f"🚨 Erro ao salvar arquivo: {e}")
        return False

# --- FUNÇÃO PRINCIPAL: GERAR ÁUDIO ---
def gerar_audio_review():
    """
    Lê as variáveis globais (texto e emoção) e gera um áudio narrado
    com a entonação apropriada usando o Gemini TTS.
    """
    # Acessa as variáveis globais definidas na etapa de análise
    texto = globals().get('review_texto_global')
    sentimento = globals().get('sentimento_global')
    emocao = globals().get('emocao_global', '') # Padrão vazio

    if not texto:
        print("🚨 Nenhuma review encontrada na memória. Rode a função 'analisar_review_steam' primeiro.")
        return

    try:
        # --- 1. DEFINIÇÃO DA PERSONA DE VOZ ---

        # Padrões
        nome_da_voz = "Kore"
        tom_da_voz = "calmo e neutro"

        # Normaliza para minúsculas para facilitar a comparação
        emo_lower = emocao.lower()
        sent_lower = sentimento.lower() if sentimento else ""

        # Lógica de Decisão (Baseada em Ekman)
        if "felicidade" in emo_lower:
            nome_da_voz = "Puck"      # Voz otimista
            tom_da_voz = "feliz e satisfeito"
        elif "raiva" in emo_lower:
            nome_da_voz = "Fenrir"    # Voz excitável
            tom_da_voz = "irritado, agressivo e frustrado"
        elif "tristeza" in emo_lower:
            nome_da_voz = "Schedar"   # Voz grave/lenta
            tom_da_voz = "triste e desanimado"
        elif "surpresa" in emo_lower:
            nome_da_voz = "Autonoe"   # Voz brilhante
            tom_da_voz = "surpreso e chocado"
        elif "medo" in emo_lower:
            nome_da_voz = "Enceladus" # Voz suave
            tom_da_voz = "assustado, nervoso e hesitante"
        elif "nojo" in emo_lower:
            nome_da_voz = "Algenib"   # Voz rouca
            tom_da_voz = "com nojo, desprezo e sarcasmo"

        # Fallback para Polaridade simples se nenhuma emoção específica for forte
        elif "positivo" in sent_lower:
            nome_da_voz = "Aoede"
            tom_da_voz = "positivo e satisfeito"
        elif "negativo" in sent_lower:
            nome_da_voz = "Orus"
            tom_da_voz = "crítico e insatisfeito"

        # --- 2. CHAMADA DA API ---
        print(f"🎭 Emoção Base: {emocao} (Sentimento: {sentimento})")
        print(f"🎙️ Gerando áudio com voz '{nome_da_voz}' e tom '{tom_da_voz}'...")

        tts_prompt = f"Leia a seguinte review de jogo de forma {tom_da_voz}: {texto}"

        tts_model = genai.GenerativeModel(model_name="gemini-2.5-flash-preview-tts")

        response = tts_model.generate_content(
            contents=tts_prompt,
            generation_config={
                "response_modalities": ["AUDIO"],
                "speech_config": {
                    "voice_config": {
                        "prebuilt_voice_config": {
                            "voice_name": nome_da_voz
                        }
                    }
                }
            }
        )

        # --- 3. REPRODUÇÃO ---
        if response.candidates and response.candidates[0].content.parts:
            audio_data = response.candidates[0].content.parts[0].inline_data.data
            if save_audio_file(audio_data, "review_audio.wav"):
                print("✅ Áudio gerado com sucesso!")
                display(Audio("review_audio.wav"))
        else:
            print("🚨 A API não retornou dados de áudio.")

    except Exception as e:
        print(f"🚨 Falha na geração de áudio: {e}")

---
### 4.3. Células de Execução das Funções Principais



### Execução do Pipeline

Agora que todas as ferramentas estão configuradas, utilize as células abaixo para testar o projeto.

1.  **Análise de Texto:** Na primeira célula, defina o `id_do_jogo` (você pode consultar a lista) e execute. O sistema buscará uma review, fará a análise de sentimentos/emoções e exibirá o resultado.

2.  **Geração de Áudio:** Logo em seguida, execute a segunda célula para ouvir a narração dessa review. O áudio será gerado com a entonação baseada na emoção detectada na etapa anterior.

In [28]:
# --- PARÂMETROS ---
id_do_jogo = 3527290  # (Exemplos: 730, 570, 1086940, 1245620)
quantidade = 100
tamanho_minimo = 30

# --- CHAMADA DA FUNÇÃO ---
# Isso fará todo o processo: busca, análise e exibição
analisar_review_steam(id_do_jogo, quantidade, tamanho_minimo)

Buscando um lote de até 100 reviews para o jogo ID: 3527290...
✅ 100 reviews obtidas com sucesso!
Buscando uma review aleatória de um total de 53 reviews filtradas...
✅ Análise concluída!



---
## Análise de Review Aleatória do Jogo: `3527290`

---

### 📝 Texto Original
> n da pra empurrar o amigo do penhasco

---

### 🤖 Resultados da Análise com Gemini

Breve Análise:
O autor da review expressa de forma concisa sua insatisfação com a ausência de uma mecânica específica no jogo: a impossibilidade de empurrar um amigo de um penhasco.

Pontos Positivos:
Nenhum ponto mencionado.

Pontos Negativos:
- Não é possível empurrar um amigo de um penhasco.

Classificação de Sentimento:
Negativo

Emoção Predominante:
Tristeza

---
**Resumo dos Dados Extraídos:**
* **Sentimento:** `Negativo`
* **Emoção:** `Tristeza`


In [29]:
# --- EXECUÇÃO ---
# Gera o áudio da última review analisada
gerar_audio_review()

🎭 Emoção Base: Tristeza (Sentimento: Negativo)
🎙️ Gerando áudio com voz 'Schedar' e tom 'triste e desanimado'...
✅ Áudio gerado com sucesso!




---


## 🎮 IDs de Jogos na Steam

Aqui está uma lista de referência rápida com os IDs de alguns jogos populares e aclamados na Steam, que podem ser usados para testar o analisador de reviews:

* **Counter-Strike 2:** `730`
* **Elden Ring:** `1245620`
* **Grand Theft Auto V:** `271590`
* **Cyberpunk 2077:** `1091500`
* **Red Dead Redemption 2:** `1174180`
* **Stardew Valley:** `413150`
* **Megabonk:** `3405340`
* **Cuphead:** `268910`
* **Lethal Company:** `1966720`
* **PEAK:** `3527290`


---
**Como encontrar o ID de qualquer jogo:**
1.  Vá para a página do jogo na Loja Steam.
2.  Olhe a URL no seu navegador.
3.  O número no meio da URL é o ID.
*Exemplo: `https://store.steampowered.com/app/`**`413150`**`/Stardew_Valley/`*

---